In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import json 
from pandas.io.json import json_normalize

df=pd.read_csv("/Users/veillonpaul-armand/Documents/Projets/Data/Carto/CP_conso.csv")

#variable of interest
var_insee=["quantile_rev","quantile_ouv","quantile_CSP+","quantile_retired","quantile_emp","CP_TUU"]

#all closed question
questions=[{"Thème":"Fiscalité","content":{"name":"Fisca_1","question":"Afin de financer les dépenses sociales, faut-il selon vous." ,"answers":["Reculer l'âge de la retraite","Augmenter le temps de travail","Augmenter les impôts",
        "Revoir les conditions d'attribution de certaines aides sociales"],"label":["a","b","c","d"]}},
         {"Thème":"Ecologie","content":[{'name':"Ecolo_0","question":"Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?","answers":["La pollution de l'air", "Les dérèglements climatiques", "L'érosion du littoral",
         "La biodiversité et la disparition de certaines espèces"],'label':["a","b","c","d"]},
        {"name":"Ecolo_2","question":"Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_4","question":"À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_8","question":"Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_10","question":"Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle(...) ?","answers":["Oui","Non","Je n'utilise pas la voiture"],"label":["a","b","c"]},
        {"name":"Ecolo_12","question":"Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?","answers":["Les transports en commun","Le covoiturage","L'auto partage","Le transport à la demande","Le vélo"],"label":["a","b","c","d","e"]}]},
        {"Thème":"Démocratie et Citoyenneté","content":[{"name":"Cit_1","question":"En dehors des élus politiques, faut-il donner un rôle plus important aux associations et aux organisations syndicales et professionnelles ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_4","question":"Le non-cumul des mandats instauré en 2017 pour les parlementaires (députés et sénateurs) est :","answers":["Une bonne chose","Une mauvaise chose","Je ne sais pas"],"label":["a","b","c"]},
        {"name":"Cit_7","question":"Pensez-vous qu'il serait souhaitable de réduire le nombre d'élus (hors députés et sénateurs) ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_10","question":"Faut-il prendre en compte le vote blanc ?","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_13","question":"Faut-il faciliter le déclenchement du référendum d'initiative partagée qui est applicable depuis 2015 ?","answers":["Oui","Non","Je ne sais pas"],"label":["a","b","c"]},
        {"name":"Cit_17","question":"Faut-il transformer le Sénat et le CESE?","answers":["Oui","Non","Je ne sais pas"],"label":["a","b","c"]}]},
        {"Thème":"Organisation de l'Etat","content":[{"name":"Etat_1","question":"Selon vous, l'Etat doit-il aujourd'hui transférer de nouvelles missions aux collectivités territoriales ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_3","question":"Estimez-vous avoir accès aux services publics dont vous avez besoin ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_6","question":"Avez-vous déjà utilisé certaines de ces nouvelles formes de services publics ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_7","question":"Si oui, en avez-vous été satisfait ? (nouvelles formes de services publics)","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_11","question":"Connaissez-vous le 'droit à l'erreur'?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_12","question":"Si oui, avez-vous déjà utilisé ce droit à l'erreur ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_15","question":"Faut-il donner plus d'autonomie aux fonctionnaires de terrain ?","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_17","question":"Faut-il revoir le fonctionnement et la formation de l'administration ?","answers":["Oui","Non"],"label":["a","b"]}]}]

          
            

In [2]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=[questions[0]["content"]]

for i in range(1):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["authorZipCode"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        df=df.merge(all_ref,how="left", right_on="authorZipCode",left_on="Code Postal")
        
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        


plot_data= pd.DataFrame()        
for lab in var_insee:
        var=sorted(list(set(df[lab])))
        plot_temp = pd.DataFrame({ "var_insee":[lab]*len(var),"quantile": var,})
        for i in range(1):
            l=len(question[i]["answers"])
            for j in range(l):
                plot_temp[question[i]["name"]+question[i]["label"][j]]=0
                for k in var:
                    plot_temp.loc[plot_temp['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df[lab]==k,question[i]["name"]+question[i]["label"][j]]*
                        df.loc[df[lab]==k,"POP_CP"])/sum(df.loc[df[lab]==k,"POP_CP"])
        plot_data=plot_data.append(plot_temp)     


In [3]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/LA_TRANSITION_ECOLOGIQUE_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=questions[1]["content"]

for i in range(len(question)):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["authorZipCode"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        df=df.merge(all_ref,how="left", right_on="authorZipCode",left_on="Code Postal")
        
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        
plot_data= pd.DataFrame()        
for lab in var_insee:
        var=sorted(list(set(df[lab])))
        plot_temp = pd.DataFrame({ "var_insee":[lab]*len(var),"quantile": var,})
        for i in range(len(question)):
            l=len(question[i]["answers"])
            for j in range(l):
                plot_temp[question[i]["name"]+question[i]["label"][j]]=0
                for k in var:
                    plot_temp.loc[plot_temp['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df[lab]==k,question[i]["name"]+question[i]["label"][j]]*
                        df.loc[df[lab]==k,"POP_CP"])/sum(df.loc[df[lab]==k,"POP_CP"])
        plot_data=plot_data.append(plot_temp)  
   

In [4]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/DEMOCRATIE_ET_CITOYENNETE_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=questions[2]["content"]

for i in range(len(question)):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["authorZipCode"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        df=df.merge(all_ref,how="left", right_on="authorZipCode",left_on="Code Postal")
        
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        
plot_data= pd.DataFrame()        
for lab in var_insee:
        var=sorted(list(set(df[lab])))
        plot_temp = pd.DataFrame({ "var_insee":[lab]*len(var),"quantile": var,})
        for i in range(len(question)):
            l=len(question[i]["answers"])
            for j in range(l):
                plot_temp[question[i]["name"]+question[i]["label"][j]]=0
                for k in var:
                    plot_temp.loc[plot_temp['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df[lab]==k,question[i]["name"]+question[i]["label"][j]]*
                        df.loc[df[lab]==k,"POP_CP"])/sum(df.loc[df[lab]==k,"POP_CP"])
        plot_data=plot_data.append(plot_temp)  
   

In [5]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')



question=questions[3]["content"]

for i in range(len(question)):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["authorZipCode"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        df=df.merge(all_ref,how="left", right_on="authorZipCode",left_on="Code Postal")
        
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        
plot_data= pd.DataFrame()        
for lab in var_insee:
        var=sorted(list(set(df[lab])))
        plot_temp = pd.DataFrame({ "var_insee":[lab]*len(var),"quantile": var,})
        for i in range(len(question)):
            l=len(question[i]["answers"])
            for j in range(l):
                plot_temp[question[i]["name"]+question[i]["label"][j]]=0
                for k in var:
                    plot_temp.loc[plot_temp['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df[lab]==k,question[i]["name"]+question[i]["label"][j]]*
                        df.loc[df[lab]==k,"POP_CP"])/sum(df.loc[df[lab]==k,"POP_CP"])
        plot_data=plot_data.append(plot_temp)  

In [23]:
##Participation par décile

df=pd.read_csv("/Users/veillonpaul-armand/Documents/Projets/Data/Carto/GN_conso_CP_level.csv")
var_part=["Count_pourmil_CP","perc_ecolo","perc_etat","perc_cit","perc_fisca"]

plot_data= pd.DataFrame()  
for lab in var_insee:
    var=sorted(list(set(df[lab])))
    plot_temp = pd.DataFrame({ "var_insee":[lab]*len(var),"quantile": var})
    for i in var_part:
        temp=df.loc[~df[i].isnull()]  
        plot_temp[i]=0
        for k in var:
            plot_temp.loc[plot_temp['quantile']==k,i]=sum(temp.loc[temp[lab]==k,i]*temp.loc[temp[lab]==k,"POP_CP"])/sum(temp.loc[temp[lab]==k,"POP_CP"])
    plot_data=plot_data.append(plot_temp)  

In [24]:
plot_data

,var_insee,quantile,Count_pourmil_CP,perc_ecolo,perc_etat,perc_cit,perc_fisca
0,quantile_rev,0.0,1.532274,0.264814,0.212235,0.207390,0.351432
1,quantile_rev,0.1,2.066140,0.263964,0.230079,0.209440,0.344924
2,quantile_rev,0.2,2.321880,0.265696,0.221455,0.200657,0.339549
3,quantile_rev,0.3,2.706356,0.256817,0.214539,0.196092,0.345997
4,quantile_rev,0.4,3.003015,0.269917,0.220541,0.193211,0.336643
5,quantile_rev,0.5,2.578893,0.267254,0.210350,0.197740,0.336095
6,quantile_rev,0.6,2.971172,0.264742,0.205513,0.194749,0.345472
7,quantile_rev,0.7,3.987481,0.261781,0.211120,0.195556,0.346536
8,quantile_rev,0.8,4.320880,0.262287,0.210067,0.190827,0.345317
9,quantile_rev,0.9,6.240630,0.257382,0.196133,0.191986,0.359303
